# Hand Replay Viewer

Replay recorded hands with interactive controls.

In [ ]:
from holdem_lab import (
    parse_cards, format_cards,
    GameState, Street,
    EventLog, EventType, HandReplayer,
)
try:
    from ipywidgets import interact, IntSlider, Output
    HAS_WIDGETS = True
except ImportError:
    HAS_WIDGETS = False
    print("Note: ipywidgets not installed. Interactive widgets disabled.")
    print("Install with: pip install ipywidgets")

## Generate a Sample Hand

In [ ]:
# Run a hand with logging enabled
game = GameState(num_players=3, seed=42, log_events=True)

# Run through each street manually to log equity
game.deal_hole_cards()
game.calculate_equity(num_simulations=1000)  # Log preflop equity

game.deal_flop()
game.calculate_equity(num_simulations=1000)  # Log flop equity

game.deal_turn()
game.calculate_equity(num_simulations=1000)  # Log turn equity

game.deal_river()
result = game.resolve()

print(f"Hand complete! Winner: Player {result.winners[0]}")
print(f"Winning hand: {result.winning_hand.describe()}")
print(f"Total events logged: {len(game.event_log)}")

## View All Events

In [ ]:
# Display all events
log = game.event_log
print("Event Log:")
print("=" * 60)

for i, event in enumerate(log):
    print(f"[{i:2}] {event.event_type.value:15} | {event.data}")

## Manual Step-by-Step Replay

In [ ]:
# Create replayer and step through
replayer = HandReplayer(log)

def display_state(state):
    """Pretty print a hand state."""
    if state is None:
        print("No state")
        return
    
    print(f"Step {state.step}: {state.event.event_type.value}")
    print("-" * 40)
    
    # Show hole cards
    if state.hole_cards:
        print("Hole Cards:")
        for player, cards in state.hole_cards.items():
            cards_str = format_cards(cards) if cards else "?"
            print(f"  Player {player}: {cards_str}")
    
    # Show board
    if state.board:
        print(f"Board: {format_cards(state.board)}")
    
    # Show equity
    if state.equities:
        print("Equity:")
        for player, eq in state.equities.items():
            print(f"  Player {player}: {eq:.1%}")
    
    # Show winners
    if state.winners is not None:
        print(f"Winners: {state.winners}")
    
    print()

In [ ]:
# Replay all states
replayer.reset()
print("Full Hand Replay")
print("=" * 60)

states = replayer.get_all_states()
for state in states:
    display_state(state)

## Interactive Slider (if ipywidgets installed)

In [ ]:
if HAS_WIDGETS:
    output = Output()
    
    def show_step(step):
        output.clear_output(wait=True)
        with output:
            state = replayer.goto_step(step)
            display_state(state)
    
    print("Use the slider to navigate through the hand:")
    interact(show_step, step=IntSlider(min=0, max=len(log)-1, step=1, value=0))
    display(output)
else:
    print("Interactive slider requires ipywidgets.")
    print("Use the manual replay above instead.")

## Save and Load Hands

In [ ]:
# Save the hand log
import tempfile
import os

# Save to temp file
temp_dir = tempfile.mkdtemp()
save_path = os.path.join(temp_dir, "sample_hand.json")

log.save(save_path)
print(f"Saved to: {save_path}")

# Load it back
loaded_log = EventLog.load(save_path)
print(f"Loaded {len(loaded_log)} events")

In [ ]:
# Verify loaded log
loaded_replayer = HandReplayer(loaded_log)
loaded_states = loaded_replayer.get_all_states()

print("Loaded hand summary:")
print(f"  Total events: {len(loaded_log)}")
print(f"  Hand ID: {loaded_log.hand_id}")

# Find the showdown event
for state in loaded_states:
    if state.event.event_type == EventType.SHOWDOWN:
        print(f"  Winners: Player {state.winners}")
        break

## Run Multiple Hands and Compare

In [ ]:
# Run multiple hands and collect stats
num_hands = 100
winners_count = {0: 0, 1: 0, 2: 0}

for seed in range(num_hands):
    game = GameState(num_players=3, seed=seed, log_events=False)
    result = game.run_to_showdown()
    for w in result.winners:
        winners_count[w] += 1 / len(result.winners)  # Split ties

print(f"Results from {num_hands} hands:")
for player, wins in winners_count.items():
    print(f"  Player {player}: {wins:.1f} wins ({wins/num_hands*100:.1f}%)")